In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
import requests
!pip install pip-system-certs

In [14]:
# Load API Credentials
with open('C:\\Users\\X474241\OneDrive - Old Mutual\\Documents\\GitHub\\yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [15]:
# set our API call parameters 
LOCATION = '50 5th Ave, New York, NY 10118'
TERM = 'pasta'

In [16]:
JSON_FILE = "Data/results_in_progress_NY_pasta.json"
JSON_FILE

'Data/results_in_progress_NY_pasta.json'

In [17]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pasta.json not found. Saving empty list to file.


In [18]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [19]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [4]:
# Load API Credentials
#with open('C:\\Users\\X474241\OneDrive - Old Mutual\\Documents\\GitHub\\yelp_api.json') as f:
#    login = json.load(f)

In [5]:
#headers = {'Authorization': 'Bearer {}'.format(login['api-key'])}

In [6]:
#params = {"term": 'pasta',
   #       "location": "50 5th Ave, New York, NY 10118",
   #       "offset": 20}

In [7]:
## Provide API Key in header
#headers = {'Authorization': 'Bearer {}'.format(login['api-key'])}
## specify query parameters
#params = {"term": 'pasta',
 #       "location": "50 5th Ave, New York, NY 10118"}
## Use requests.get and save the response
#response = requests.get(search_url, headers=headers, params=params)
#response

<Response [200]>

In [20]:
## How many results total?
total_results = results['total']
total_results

3000

In [21]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [22]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

150

In [23]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [24]:
!pip install tqdm

In [25]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/150 [00:00<?, ?it/s]

In [26]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/150 [00:00<?, ?it/s]

HTTPError: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=50+5th+Ave%2C+New+York%2C+NY+10118&term=pasta&offset=1000

In [27]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [28]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [29]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NY_pasta.json not found. Saving empty list to new file.
- 0 previous results found.


150

In [30]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/150 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [31]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,16ZnHpuaaBt92XWeJHCC5A,olio-e-più-new-york-7,Olio e Più,https://s3-media4.fl.yelpcdn.com/bphoto/CUpPgz...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,4897,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.733798036104304, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '3 Greenwich Ave', 'address2': No...",+12122436546,(212) 243-6546,392.670772
1,hdiuRS9sVZSMReZm4oV5SA,da-andrea-new-york,Da Andrea,https://s3-media1.fl.yelpcdn.com/bphoto/OciLdd...,False,https://www.yelp.com/biz/da-andrea-new-york?ad...,2101,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.736218, 'longitude': -73.99597}","[restaurant_reservation, delivery, pickup]",$$,"{'address1': '35 W 13th St', 'address2': '', '...",+12123671979,(212) 367-1979,229.670682
2,R-ve5GRzP0WlbQvDVI3HKg,pane-pasta-new-york,Pane Pasta,https://s3-media4.fl.yelpcdn.com/bphoto/PoKPQW...,False,https://www.yelp.com/biz/pane-pasta-new-york?a...,64,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 40.73328, 'longitude': -73.99905}","[delivery, pickup]",NaN,"{'address1': '58 West 8th St', 'address2': Non...",+16468910006,(646) 891-0006,348.312262
3,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3194,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1279.967959
4,hSpZSqElyjHTfE4lQJ97Cg,mareluna-new-york,Mareluna,https://s3-media1.fl.yelpcdn.com/bphoto/Sec0CN...,False,https://www.yelp.com/biz/mareluna-new-york?adj...,82,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.726787, 'longitude': -74.001836}","[delivery, pickup]",NaN,"{'address1': '137 Sullivan St', 'address2': No...",+12125107774,(212) 510-7774,1002.742646


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,mXN9tmuDHZIfCeUBYqSRuQ,john-doe-new-york,John Doe,https://s3-media2.fl.yelpcdn.com/bphoto/gM7oSy...,False,https://www.yelp.com/biz/john-doe-new-york?adj...,208,"[{'alias': 'newamerican', 'title': 'New Americ...",4.0,"{'latitude': 40.74477, 'longitude': -73.98701}","[pickup, delivery]",$$,"{'address1': '253 5th Ave', 'address2': '', 'a...",+16468824007,(646) 882-4007,1356.970555
996,-KKgqE2g4c1cH4YmmDR9CQ,bernies-brooklyn,Bernie's,https://s3-media1.fl.yelpcdn.com/bphoto/hWlvgX...,False,https://www.yelp.com/biz/bernies-brooklyn?adju...,132,"[{'alias': 'newamerican', 'title': 'New Americ...",4.5,"{'latitude': 40.721936, 'longitude': -73.950597}",[],$$,"{'address1': '332 Driggs Ave', 'address2': '',...",+13475296400,(347) 529-6400,3996.944798
997,BaeUVrAj5f3YF1razW5bow,11-tigers-new-york-2,11 Tigers,https://s3-media2.fl.yelpcdn.com/bphoto/tU8h7o...,False,https://www.yelp.com/biz/11-tigers-new-york-2?...,157,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 40.72908, 'longitude': -73.97815}","[pickup, delivery, restaurant_reservation]",NaN,"{'address1': '225 Avenue B', 'address2': 'Fl 2...",+19292356826,(929) 235-6826,1546.159588
998,G-b9kvnRh0pLi4fsdcmwjg,josephs-restaurant-new-york-112,Joseph's Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/ZGG37M...,False,https://www.yelp.com/biz/josephs-restaurant-ne...,91,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.70499, 'longitude': -74.00961}","[pickup, delivery]",$$,"{'address1': '3 Hanover Square', 'address2': N...",+12127471300,(212) 747-1300,3484.237934
999,4M3jBdELCcCKJl1mEnTxzQ,lamias-fish-market-new-york-2,Lamia's Fish Market,https://s3-media4.fl.yelpcdn.com/bphoto/15pOL7...,False,https://www.yelp.com/biz/lamias-fish-market-ne...,742,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.5,"{'latitude': 40.722979, 'longitude': -73.9825102}","[pickup, delivery]",$$$,"{'address1': '47 Ave B', 'address2': None, 'ad...",+12127773650,(212) 777-3650,1644.620271


In [33]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [34]:
# save the final results to a compressed csv
final_df.to_csv('Data/results_in_progress_NY_pasta.csv.gz', compression='gzip',index=False)